In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import os.path
import datetime
import pandas as pd
import sys
sys.path.append('../libraries/backtrader')
sys.path.append('../')
import backtrader as bt


In [2]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()


In [3]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Datas are in a subfolder of the samples
datapath = '/home/fcelaya/quant3/data/us/daily/aapl.csv'
df = pd.read_csv(datapath)
df['date'] = pd.to_datetime(df['date'])

# Create a Data Feed
data = bt.feeds.PandasData(dataname=df,datetime=0,open=1,high=2,low=3,close=4,volume=5)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Set the commission - 0.1% ... divide by 100 to remove the %
cerebro.broker.setcommission(commission=0.001)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2010-01-25, Close, 7.25
2010-01-26, Close, 7.36
2010-01-27, Close, 7.42
2010-01-28, Close, 7.12
2010-01-29, Close, 6.86
2010-02-01, Close, 6.95
2010-02-02, Close, 6.99
2010-02-03, Close, 7.12
2010-02-04, Close, 6.86
2010-02-05, Close, 6.98
2010-02-08, Close, 6.93
2010-02-09, Close, 7.01
2010-02-10, Close, 6.97
2010-02-11, Close, 7.10
2010-02-11, BUY CREATE, 7.10
2010-02-12, BUY EXECUTED, Price: 7.08, Cost: 70.75, Comm 0.07
2010-02-12, Close, 7.16
2010-02-16, Close, 7.26
2010-02-17, Close, 7.23
2010-02-18, Close, 7.25
2010-02-19, Close, 7.20
2010-02-22, Close, 7.16
2010-02-23, Close, 7.04
2010-02-23, SELL CREATE, 7.04
2010-02-24, SELL EXECUTED, Price: 7.08, Cost: 70.75, Comm 0.07
2010-02-24, OPERATION PROFIT, GROSS 0.04, NET -0.10
2010-02-24, Close, 7.17
2010-02-24, BUY CREATE, 7.17
2010-02-25, BUY EXECUTED, Price: 7.05, Cost: 70.49, Comm 0.07
2010-02-25, Close, 7.21
2010-02-26, Close, 7.31
2010-03-01, Close, 7.46
2010-03-02, Close, 7.46
2010-03-03, C